# PRUEBA VGG16

https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [2]:
pip install tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


Primero debemos descagar el set de imágenes con las que probaremos VGG16, son imágenes de distintos tipos de flores. Están en el paquete tfds y para incluirlas en el algoritmo del modelo debemos redimensionarla con resize(), y, posteriormente, convertir su información en datos categóricos con to_categorical(), una función de tensorflow. Esta función devuelve una matriz de valores binarios (ya sea '1' o '0'), tiene un número de filas igual a la longitud del vector de entrada y un número de columnas igual al número de clases. De tal manera que cada imagen seria un elemento del vector de entrada (una fila de la matriz) y quedará reflejado un 1 en la columna que corresponda con la clase a la que pertenece.

https://www.geeksforgeeks.org/python-keras-keras-utils-to_categorical/

In [3]:
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

Después nos descargamos la red VGG16 y no cambiamos los pesos iniciales que recoge de "imagenet". Su estructura tiene los siguientes componentes.

Input

Convolution(x2)
MaxPooling

Convolution(x2)
MaxPooling

Convolution(x3)
MaxPooling

Convolution(x3)
MaxPooling

Convolution(x3)
MaxPooling

Flatten
Dense(x3)

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

Aquí queda mejor resumido el contenido de VGG16.

In [6]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

Las últimas capas para clasificar estas flores en concreto debemos adaptarlas a nuestro caso particular. Por eso debemos definir las capas densas y una última capa de predicción. En la variable "model" queda almacenado el modelo VGG16 básico junto con estas últimas capas.

In [7]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

Ya podemos compilar el modelo y entrenarlo con nuestros datos de interés, que se encuentran en train_ds y train_labels. Utilizamos el fit() junto con EarlyStopping para que no se ejecuten todas las iteraciones marcadas en caso de la métrica ya no pueda mejorar más.

https://keras.io/api/callbacks/early_stopping/

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 100s 2s/step - loss: 1.7899 - accuracy: 0.4813 - val_loss: 1.1998 - val_accuracy: 0.5389
Epoch 2/50
65/65 [==============================] - 96s 1s/step - loss: 0.9098 - accuracy: 0.7012 - val_loss: 1.2066 - val_accuracy: 0.5311
Epoch 3/50
65/65 [==============================] - 97s 2s/step - loss: 0.6878 - accuracy: 0.7655 - val_loss: 1.0042 - val_accuracy: 0.6712
Epoch 4/50
65/65 [==============================] - 102s 2s/step - loss: 0.4897 - accuracy: 0.8190 - val_loss: 1.0202 - val_accuracy: 0.7004
Epoch 5/50
65/65 [==============================] - 98s 2s/step - loss: 0.3492 - accuracy: 0.8720 - val_loss: 1.0528 - val_accuracy: 0.7004
Epoch 6/50
65/65 [==============================] - 99s 2s/step - loss: 0.2521 - accuracy: 0.9022 - val_loss: 1.2829 - val_accuracy: 0.6809
Epoch 7/50
65/65 [==============================] - 104s 2s/step - loss: 0.2048 - accuracy: 0.9236 - val_loss: 1.3340 - val_accuracy: 0.6770
Epoch 8/50
65/65 